In [1]:
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install scipy
!pip install statsmodels
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-pac

In [3]:
pip show scikit-learn

Name: scikit-learn
Version: 1.4.2
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /home/vscode/.local/lib/python3.10/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: 
Note: you may need to restart the kernel to use updated packages.


Logistic regression

In [ ]:
import pandas as pd
from scikit-learn.linear_model import LogisticRegression
from scikit-learn.preprocessing import StandardScaler

df = pd.DataFrame({'x':[1,2,3,4,5,6,7,8,9,10],'y':[0,0,0,0,1,0,1,1,1,1]})

logit = LogisticRegression(fit_intercept = True, penalty = None, class_weight = 'balanced')
scaler = StandardScaler()
df["x scaled"] = scaler.fit_transform(df[["x"]])
logit.fit(df[["x scaled"]], df["y"])

p = [i[1] for i in logit.predict_proba(df[["x scaled"]])]

df['Predicted Probability'] = p # logit.predict(df[['x']])
df = df.loc[:,["x","y","Predicted Probability"]]
df.head(10)

In [ ]:
!pip install pandas asdf

Anomaly detection

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from matplotlib import pyplot

df = pd.read_csv("https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_station/USW00023190.csv.gz",
                 compression = 'gzip',
                 names = ['station','date','datatype','value','U1','U2','U3','U4'],
                 low_memory = False)

df["date"] = pd.to_datetime(df["date"].astype(str))
df = df.loc[(df["datatype"].isin(["TMIN","TMAX","PRCP"])) & (df["date"] >= "2019-01-01"), ["date", "datatype", "value"]]

df["value"] = np.where(np.array(df["datatype"]) == "PRCP", np.round(np.array(df["value"])/100/2.54, 2), np.round(np.array(df["value"])/10 * 1.8 + 32, 0))
df = df.pivot(index = "date", columns = "datatype", values = "value").dropna().rename_axis(None, axis = 1).reset_index(inplace = False)

df["TAVG"] = (df["TMIN"] + df["TMAX"]) / 2
df = df.rename(columns = {"date": "Date", "TMAX": "High Temperature", "TMIN": "Low Temperature", "TAVG": "Average Temperature", "PRCP": "Rainfall"})

X = np.array(df["Average Temperature"]).reshape(-1,1)

eps = 2
min_samples = 14

clusters = DBSCAN(eps = eps, min_samples = min_samples).fit_predict(X)
df = df.assign(Anomaly = clusters)
df.tail()

Time series decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import pyplot
series = df["Average Temperature"]
result = seasonal_decompose(series, model = 'additive', period = 365)
result.plot()
pyplot.show()

In [ ]:
df.dtypes

In [ ]:
monthly = df.groupby(pd.Grouper(key = 'Date', axis = 0, freq = 'M')).mean()
monthly.head(40)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import pyplot
series = monthly["Average Temperature"]
result = seasonal_decompose(series, model = 'additive', period = 12)
result.plot()
pyplot.show()

Clustering

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage

temps = [["Bakersfield", 78,  53, 65.5],
        ["Burbank", 77, 55, 66],
        ["Fresno", 77,  51, 64],
        ["Long Beach", 74,  55, 64.5],
        ["Los Angeles", 70,  56, 63],
        ["Sacramento", 74,  48, 61],
        ["San Diego", 71,  57, 64],
        ["San Francisco", 66,  50, 58],
        ["Santa Barbara", 70,  49, 59.5]]
df = pd.DataFrame(temps, columns = ["Station", "High Temperature", "Low Temperature", "Average Temperature"])
data_1d = list(zip(df["Average Temperature"].tolist()))
data_2d = list(zip(df["Low Temperature"].tolist(), df["High Temperature"].tolist()))

df = df.set_index("Station")

df.head(10)

In [ ]:
distances = linkage(data_1d, metric = "euclidean")
dendrogram(distances, leaf_rotation = 0, leaf_font_size = 10, labels = df.index, orientation = "right") # can change threshold colors as well
plt.show()

In [ ]:
distances = linkage(data_2d, method = "ward", metric = "euclidean")
dendrogram(distances, leaf_rotation = 0, leaf_font_size = 10, labels = df.index, orientation = "right") # can change threshold colors as well
plt.show()

Seaborn dendrogram

In [ ]:
df.head(10)

In [ ]:
sns.clustermap(df, cmap = "Oranges")
plt.show()

In [ ]:
df = pd.DataFrame([[1, 2], [3, 4]])
print(df)